In [35]:
# !pip install -r ../exp_requirements.txt

In [36]:
# !pip uninstall virny -y

In [37]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@development

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@development

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /home/dh3553/projects/fairness-variance


# Folktables GA Dataset With Outliers

## Import dependencies

In [4]:
import os
import copy

from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from virny.utils.custom_initializers import create_config_obj, create_models_metrics_dct_from_database_df
from virny.datasets.data_loaders import ACSIncomeDataset
from source.user_interfaces.experiment_interface import run_exp_iter_with_models_stress_testing
from source.error_injectors.outliers_injector_v2 import OutliersInjectorV2
from source.utils.custom_initializers import create_experiment_data_loader
from source.utils.db_functions import read_model_metric_dfs_from_db
from source.preprocessing.basic_preprocessing import get_null_imputer_preprocessor

from configs.constants import NUM_METRICS_COMPUTATION_RUNS, EXPERIMENT_SEEDS, TEST_SET_FRACTION
from configs.models_params_for_tuning import get_folktables_income_models_params_for_tuning

## Initialize input variables for the experiment

In [5]:
# ROOT_DIR = os.path.join(os.getcwd(), "..", "..")
ROOT_DIR = os.getcwd()
EXPERIMENT_NAME = 'stress_testing_outliers'
DB_COLLECTION_NAME = f'exp_{EXPERIMENT_NAME}'
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME)
COLUMNS_TO_TRANSFORM = ['WKHP']
INJECTOR_CONFIG_LST = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
MAX_NUM_COLUMNS_TO_EFFECT = 1

In [6]:
data_loader = ACSIncomeDataset(state=['GA'], year=2018, with_nulls=False,
                               subsample_size=20_000, subsample_seed=42)
data_loader.X_data.head()

,SCHL,COW,MAR,OCCP,POBP,RELP,SEX,RAC1P,AGEP,WKHP
0,23,7,3,230,36,0,1,1,55,55.0
1,16,1,5,4110,13,2,2,1,20,35.0
2,16,4,3,4130,51,0,2,1,59,30.0
3,18,4,1,4020,13,0,1,2,43,40.0
4,14,1,1,8300,20,1,2,2,33,20.0


In [7]:
data_loader.full_df.shape

(20000, 10)

In [8]:
data_loader.full_df.isna().sum()

SCHL     0
COW      0
MAR      0
OCCP     0
POBP     0
RELP     0
SEX      0
RAC1P    0
AGEP     0
WKHP     0
dtype: int64

### Define a db writer and custom fields to insert into your database

In [9]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [10]:
custom_table_fields_dct = {
    'error_type': 'Random Outliers',
    'error_percentages': '0%,5%,10%,20%,30%,40%,50%',
    'preprocessing_technique': 'No',
}

In [11]:
import uuid

custom_table_fields_dct['session_uuid'] = '715ef4e8-fd23-4bd1-ad30-8fe69a7068d1'
# custom_table_fields_dct['session_uuid'] = str(uuid.uuid4())
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  715ef4e8-fd23-4bd1-ad30-8fe69a7068d1


### Create a metrics computation config object

In [12]:
config_yaml_path = 'experiment_config.yaml'
age_range = [i for i in range(30, 46)]
config_yaml_content = \
f"""
dataset_name: Folktables_Income_GA_2018
bootstrap_fraction: 0.8
n_estimators: 200
num_runs: 1
runs_seed_lst: [100]
sensitive_attributes_dct: {{'SEX': '1', 'RAC1P': '1', 'AGEP': {age_range}, 'SEX & RAC1P': None, 'RAC1P & AGEP': None}}
"""

with open(config_yaml_path, 'w', encoding='utf-8') as f:
    f.write(config_yaml_content)

In [13]:
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

### Define tuning parameter for models

In [14]:
# def get_models_params_for_tuning(models_tuning_seed):
#     return {
#         'DecisionTreeClassifier': {
#             'model': DecisionTreeClassifier(random_state=models_tuning_seed),
#             'params': {
#                 "max_depth": [20, 30],
#                 "min_samples_split" : [0.1],
#                 "max_features": ['sqrt'],
#                 "criterion": ["gini", "entropy"]
#             }
#         },
#         'LogisticRegression': {
#             'model': LogisticRegression(random_state=models_tuning_seed),
#             'params': {
#                 'penalty': ['l2'],
#                 'C' : [0.0001, 0.1, 1, 100],
#                 'solver': ['newton-cg', 'lbfgs'],
#                 'max_iter': [250],
#             }
#         },
#         # 'RandomForestClassifier': {
#         #     'model': RandomForestClassifier(random_state=models_tuning_seed),
#         #     'params': {
#         #         "max_depth": [6, 10],
#         #         "min_samples_leaf": [1],
#         #         "n_estimators": [50, 100],
#         #         "max_features": [0.6]
#         #     }
#         # },
#         # 'XGBClassifier': {
#         #     'model': XGBClassifier(random_state=models_tuning_seed, verbosity=0),
#         #     'params': {
#         #         'learning_rate': [0.1],
#         #         'n_estimators': [200],
#         #         'max_depth': [5, 7],
#         #         'lambda':  [10, 100]
#         #     }
#         # },
#         # 'KNeighborsClassifier': {
#         #     'model': KNeighborsClassifier(),
#         #     'params': {
#         #         'n_neighbors' : [5, 7, 9, 11, 13, 15, 25],
#         #         'weights' : ['uniform', 'distance'],
#         #         'metric' : ['minkowski', 'euclidean', 'manhattan']
#         #     }
#         # },
#         # 'MLPClassifier': {
#         #     'model': MLPClassifier(random_state=models_tuning_seed),
#         #     'params': {
#         #         'hidden_layer_sizes':[(100,), (100,100,), (100,50,100,)],
#         #         'activation': ['logistic', 'tanh', 'relu'],
#         #         'solver': ['lbfgs', 'sgd', 'adam'],
#         #         'learning_rate': ['constant', 'invscaling', 'adaptive']
#         #     }
#         # }
#     }

## Run experiments

### Experiment iteration 1

In [34]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filename = 'tuning_results_Folktables_Income_GA_2018_exp_iter_1_20230425__235953.csv'
tuned_params_df_path = os.path.join(ROOT_DIR, 'results', 'stress_testing_nulls',
                                    'models_tuning', tuned_params_filename)
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

error_injector = OutliersInjectorV2(experiment_seed, columns_to_transform=COLUMNS_TO_TRANSFORM,
                                    row_idx_percentage=0.0,
                                    max_num_columns_to_effect=MAX_NUM_COLUMNS_TO_EFFECT)
models_params_for_tuning = get_folktables_income_models_params_for_tuning(experiment_seed)
# Create a transformed data loader
exp_iter_data_loader = create_experiment_data_loader(data_loader)
exp_iter_data_loader.columns_with_nulls = COLUMNS_TO_TRANSFORM
preprocessor = get_null_imputer_preprocessor(exp_iter_data_loader)

In [ ]:
run_exp_iter_with_models_stress_testing(data_loader=exp_iter_data_loader,
                                        experiment_seed=experiment_seed,
                                        test_set_fraction=TEST_SET_FRACTION,
                                        db_writer_func=db_writer_func,
                                        error_injector=error_injector,
                                        injector_config_lst=INJECTOR_CONFIG_LST,
                                        preprocessor=preprocessor,
                                        models_params_for_tuning=models_params_for_tuning,
                                        metrics_computation_config=metrics_computation_config,
                                        custom_table_fields_dct=custom_table_fields_dct,
#                                         with_tuning=True,
#                                         tuned_params_df_path=None,
                                        with_tuning=False,
                                        tuned_params_df_path=tuned_params_df_path,
                                        save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                        verbose=True)

2023-04-26 04:28:40 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:


{'dataset_split_seed': 100,
 'error_percentages': '0%,5%,10%,20%,30%,40%,50%',
 'error_type': 'Random Outliers',
 'experiment_iteration': 'Exp_iter_1',
 'injector_config_lst': '[0.05, 0.1, 0.2, 0.3, 0.4, 0.5]',
 'model_init_seed': 100,
 'preprocessing_technique': 'No',
 'session_uuid': '715ef4e8-fd23-4bd1-ad30-8fe69a7068d1'}




2023-04-26 04:28:41 experiment_interface.py INFO    : The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([ 2917,  2234, 14396,  1781, 11102,   732,  7692, 10589, 16098,
             4920,  6601,  7611,  3825, 18755,  6862,  3847,  7256, 13711,
            12389,  9772],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([ 2917,  2234, 14396,  1781, 11102,   732,  7692, 10589, 16098,
             4920,  6601,  7611,  3825, 18755,  6862,  3847,  7256, 13711,
            12389,  9772],
           dtype='int64')
error_injector.seed --  101
transformed_X_test:
 SCHL     0
COW      0
MAR      0
OCCP     0
POBP     0
RELP     0
SEX      0
RAC1P    0
AGEP     0
WKHP     0
dtype: int64


WKHP: 114


error_injector.seed --  102
transformed_X_test:
 SCHL     0
COW      0
MAR      0
OCCP     0
POBP     0
RELP     0
SEX      0
RAC1P    0
AGEP     0
WKHP     0
dtype: int64


WKHP: 213


error_injector.seed --  103
transformed_X_test:
 SCHL     0
COW      0
MAR      0
OCCP     0
POBP     0
RELP     0
SEX    

2023-04-26 04:28:43 experiment_interface.py INFO    : Models config is loaded from the input file


transformed_X_test:
 SCHL     0
COW      0
MAR      0
OCCP     0
POBP     0
RELP     0
SEX      0
RAC1P    0
AGEP     0
WKHP     0
dtype: int64


WKHP: 1007


DecisionTreeClassifier:  {'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 20, 'max_features': 0.6, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 0.01, 'min_weight_fraction_leaf': 0.0, 'random_state': 100, 'splitter': 'best'}
LogisticRegression:  {'C': 1, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 50, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': 100, 'solver': 'newton-cg', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


Multiple runs progress:   0%|          | 0/1 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/5 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 2

In [26]:
# Configs for an experiment iteration
exp_iter_num = 2
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filename = 'tuning_results_Folktables_Income_GA_2018_exp_iter_2_20230426__030350.csv'
tuned_params_df_path = os.path.join(ROOT_DIR, 'results', 'stress_testing_nulls',
                                    'models_tuning', tuned_params_filename)
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

error_injector = OutliersInjectorV2(experiment_seed, columns_to_transform=COLUMNS_TO_TRANSFORM,
                                    row_idx_percentage=0.0,
                                    max_num_columns_to_effect=MAX_NUM_COLUMNS_TO_EFFECT)
models_params_for_tuning = get_folktables_income_models_params_for_tuning(experiment_seed)
# Create a transformed data loader
exp_iter_data_loader = create_experiment_data_loader(data_loader)
exp_iter_data_loader.columns_with_nulls = COLUMNS_TO_TRANSFORM
preprocessor = get_null_imputer_preprocessor(exp_iter_data_loader)

In [27]:
run_exp_iter_with_models_stress_testing(data_loader=exp_iter_data_loader,
                                        experiment_seed=experiment_seed,
                                        test_set_fraction=TEST_SET_FRACTION,
                                        db_writer_func=db_writer_func,
                                        error_injector=error_injector,
                                        injector_config_lst=INJECTOR_CONFIG_LST,
                                        preprocessor=preprocessor,
                                        models_params_for_tuning=models_params_for_tuning,
                                        metrics_computation_config=metrics_computation_config,
                                        custom_table_fields_dct=custom_table_fields_dct,
#                                         with_tuning=True,
#                                         tuned_params_df_path=None,
                                        with_tuning=False,
                                        tuned_params_df_path=tuned_params_df_path,
                                        save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                        verbose=True)

2023-04-25 22:54:27 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:


{'dataset_split_seed': 200,
 'error_percentages': '0%,5%,10%,20%,30%,40%,50%',
 'error_type': 'Random Nulls',
 'experiment_iteration': 'Exp_iter_2',
 'injector_config_lst': '[0.05, 0.1, 0.2, 0.3, 0.4, 0.5]',
 'model_init_seed': 200,
 'preprocessing_technique': 'cat: mode, num: median',
 'session_uuid': '340a6ce1-3261-4b64-a083-d74dfaad97cf'}




2023-04-25 22:54:27 experiment_interface.py INFO    : The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([ 4298,  9201,   679, 18780,  6955,  8571,  9405, 13699,  3930,
            13573,  3774,  9085, 12646, 16782, 13102, 18789,  8042, 19647,
            13861,  8780],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([ 4298,  9201,   679, 18780,  6955,  8571,  9405, 13699,  3930,
            13573,  3774,  9085, 12646, 16782, 13102, 18789,  8042, 19647,
            13861,  8780],
           dtype='int64')
error_injector.seed --  201
transformed_X_test:
 SCHL     35
COW      22
MAR      23
OCCP     29
POBP     28
RELP     26
SEX       0
RAC1P     0
AGEP      0
WKHP     29
dtype: int64
error_injector.seed --  202
transformed_X_test:
 SCHL     67
COW      52
MAR      62
OCCP     60
POBP     52
RELP     54
SEX       0
RAC1P     0
AGEP      0
WKHP     61
dtype: int64
error_injector.seed --  203
transformed_X_test:
 SCHL     117
COW      108
MAR      103
OCCP     119
POBP     118
RELP     121
SEX

2023-04-25 23:03:50 experiment_interface.py INFO    : Models are tuned and saved to a file


...

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x1479a38acdc0>
Traceback (most recent call last):
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x1479a38acdc0>
Traceback (most recent call last):
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x1479a7b14c10>
Traceback (most recent call last):
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x1479a7b14c10>
Traceback (most recent call last):
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x1479a3774af0>
Traceback (most recent call last):
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x

### Experiment iteration 3

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 3
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filename = 'tuning_results_Folktables_Income_GA_2018_exp_iter_3_20230426__003144.csv'
tuned_params_df_path = os.path.join(ROOT_DIR, 'results', 'stress_testing_nulls',
                                    'models_tuning', tuned_params_filename)
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

error_injector = OutliersInjectorV2(experiment_seed, columns_to_transform=COLUMNS_TO_TRANSFORM,
                                    row_idx_percentage=0.0,
                                    max_num_columns_to_effect=MAX_NUM_COLUMNS_TO_EFFECT)
models_params_for_tuning = get_folktables_income_models_params_for_tuning(experiment_seed)
# Create a transformed data loader
exp_iter_data_loader = create_experiment_data_loader(data_loader)
exp_iter_data_loader.columns_with_nulls = COLUMNS_TO_TRANSFORM
preprocessor = get_null_imputer_preprocessor(exp_iter_data_loader)

In [ ]:
run_exp_iter_with_models_stress_testing(data_loader=exp_iter_data_loader,
                                        experiment_seed=experiment_seed,
                                        test_set_fraction=TEST_SET_FRACTION,
                                        db_writer_func=db_writer_func,
                                        error_injector=error_injector,
                                        injector_config_lst=INJECTOR_CONFIG_LST,
                                        preprocessor=preprocessor,
                                        models_params_for_tuning=models_params_for_tuning,
                                        metrics_computation_config=metrics_computation_config,
                                        custom_table_fields_dct=custom_table_fields_dct,
#                                         with_tuning=True,
#                                         tuned_params_df_path=None,
                                        with_tuning=False,
                                        tuned_params_df_path=tuned_params_df_path,
                                        save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                        verbose=True)

### Experiment iteration 4

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 4
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filename = 'tuning_results_Folktables_Income_GA_2018_exp_iter_4_20230426__034625.csv'
tuned_params_df_path = os.path.join(ROOT_DIR, 'results', 'stress_testing_nulls',
                                    'models_tuning', tuned_params_filename)
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

error_injector = OutliersInjectorV2(experiment_seed, columns_to_transform=COLUMNS_TO_TRANSFORM,
                                    row_idx_percentage=0.0,
                                    max_num_columns_to_effect=MAX_NUM_COLUMNS_TO_EFFECT)
models_params_for_tuning = get_folktables_income_models_params_for_tuning(experiment_seed)
# Create a transformed data loader
exp_iter_data_loader = create_experiment_data_loader(data_loader)
exp_iter_data_loader.columns_with_nulls = COLUMNS_TO_TRANSFORM
preprocessor = get_null_imputer_preprocessor(exp_iter_data_loader)

In [ ]:
run_exp_iter_with_models_stress_testing(data_loader=exp_iter_data_loader,
                                        experiment_seed=experiment_seed,
                                        test_set_fraction=TEST_SET_FRACTION,
                                        db_writer_func=db_writer_func,
                                        error_injector=error_injector,
                                        injector_config_lst=INJECTOR_CONFIG_LST,
                                        preprocessor=preprocessor,
                                        models_params_for_tuning=models_params_for_tuning,
                                        metrics_computation_config=metrics_computation_config,
                                        custom_table_fields_dct=custom_table_fields_dct,
#                                         with_tuning=True,
#                                         tuned_params_df_path=None,
                                        with_tuning=False,
                                        tuned_params_df_path=tuned_params_df_path,
                                        save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                        verbose=True)

### Experiment iteration 5

In [15]:
# Configs for an experiment iteration
exp_iter_num = 5
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filename = 'tuning_results_Folktables_Income_GA_2018_exp_iter_5_20230426__003352.csv'
tuned_params_df_path = os.path.join(ROOT_DIR, 'results', 'stress_testing_nulls',
                                    'models_tuning', tuned_params_filename)
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

error_injector = OutliersInjectorV2(experiment_seed, columns_to_transform=COLUMNS_TO_TRANSFORM,
                                    row_idx_percentage=0.0,
                                    max_num_columns_to_effect=MAX_NUM_COLUMNS_TO_EFFECT)
models_params_for_tuning = get_folktables_income_models_params_for_tuning(experiment_seed)
# Create a transformed data loader
exp_iter_data_loader = create_experiment_data_loader(data_loader)
exp_iter_data_loader.columns_with_nulls = COLUMNS_TO_TRANSFORM
preprocessor = get_null_imputer_preprocessor(exp_iter_data_loader)

In [16]:
run_exp_iter_with_models_stress_testing(data_loader=exp_iter_data_loader,
                                        experiment_seed=experiment_seed,
                                        test_set_fraction=TEST_SET_FRACTION,
                                        db_writer_func=db_writer_func,
                                        error_injector=error_injector,
                                        injector_config_lst=INJECTOR_CONFIG_LST,
                                        preprocessor=preprocessor,
                                        models_params_for_tuning=models_params_for_tuning,
                                        metrics_computation_config=metrics_computation_config,
                                        custom_table_fields_dct=custom_table_fields_dct,
#                                         with_tuning=True,
#                                         tuned_params_df_path=None,
                                        with_tuning=False,
                                        tuned_params_df_path=tuned_params_df_path,
                                        save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                        verbose=True)

2023-04-26 04:47:59 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:


{'dataset_split_seed': 500,
 'error_percentages': '0%,5%,10%,20%,30%,40%,50%',
 'error_type': 'Random Outliers',
 'experiment_iteration': 'Exp_iter_5',
 'injector_config_lst': '[0.05, 0.1, 0.2, 0.3, 0.4, 0.5]',
 'model_init_seed': 500,
 'preprocessing_technique': 'No',
 'session_uuid': '715ef4e8-fd23-4bd1-ad30-8fe69a7068d1'}




2023-04-26 04:47:59 experiment_interface.py INFO    : The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([  851, 17635,  1369, 18670, 19789,  1799,  8446, 14863,  4634,
             7378,  9366, 16042,  1195,   800,  1758,   548, 17791,  2416,
             6478, 11097],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([  851, 17635,  1369, 18670, 19789,  1799,  8446, 14863,  4634,
             7378,  9366, 16042,  1195,   800,  1758,   548, 17791,  2416,
             6478, 11097],
           dtype='int64')
error_injector.seed --  501
transformed_X_test:
 SCHL     0
COW      0
MAR      0
OCCP     0
POBP     0
RELP     0
SEX      0
RAC1P    0
AGEP     0
WKHP     0
dtype: int64


WKHP: 117


error_injector.seed --  502
transformed_X_test:
 SCHL     0
COW      0
MAR      0
OCCP     0
POBP     0
RELP     0
SEX      0
RAC1P    0
AGEP     0
WKHP     0
dtype: int64


WKHP: 218


error_injector.seed --  503
transformed_X_test:
 SCHL     0
COW      0
MAR      0
OCCP     0
POBP     0
RELP     0
SEX    

2023-04-26 04:48:02 experiment_interface.py INFO    : Models config is loaded from the input file


...

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x14b4c850ca60>
Traceback (most recent call last):
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x14b4ca9b6dc0>
Traceback (most recent call last):
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x14b4ca947d30>
Traceback (most recent call last):
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x14b4ca9b6dc0>
Traceback (most recent call last):
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x14b4ca9b6dc0>
Traceback (most recent call last):
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/ext3/miniconda3/lib/python3.8/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x

In [54]:
# client.close()